In [5]:
!ls

sample_data  나이분류.zip


In [1]:
import zipfile
import os

# 압축 파일 경로
zip_path = './나이분류.zip'

# 압축을 풀 폴더 경로
extract_path = './'

# 압축 풀기
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# 압축이 풀린 파일 목록 확인
os.listdir(extract_path)


['.config', '나이분류', '나이분류.zip', 'sample_data']

In [15]:
!ls -al

total 341556
drwxr-xr-x 5 root root      4096 Sep 23 00:16 나이분류
drwxr-xr-x 1 root root      4096 Sep 23 00:16 .
drwxr-xr-x 1 root root      4096 Sep 23 00:00 ..
drwxr-xr-x 4 root root      4096 Sep 19 13:25 .config
drwxr-xr-x 1 root root      4096 Sep 19 13:25 sample_data
-rw-r--r-- 1 root root 349728946 Sep 23 00:15 나이분류.zip


In [2]:
cd 나이분류

/content/나이분류


In [3]:
pwd

'/content/나이분류'

In [6]:
import os
import torch
import torchvision.models as models
import torchvision.transforms as T
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm  # TQDM import

# Custom dataset
class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = os.listdir(root_dir)
        self.data = []

        for label in range(len(self.classes)):
            class_folder = os.path.join(root_dir, self.classes[label])
            for filename in os.listdir(class_folder):
                img_path = os.path.join(class_folder, filename)
                self.data.append((img_path, label))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        image = Image.open(img_path).convert('RGB')  # 이미지 RGB로 변환
        if self.transform:
            image = self.transform(image)
        return image, label

# 경로 및 배치 크기 설정
data_dir = "."
batch_size = 32

# 데이터 증강 포함한 이미지 전처리
transform = T.Compose([
    T.Resize((224, 224)),
    T.RandomHorizontalFlip(),  # 랜덤 가로 뒤집기
    T.RandomRotation(10),  # 랜덤 회전
    T.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),  # 색상 변형
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 학습 및 검증 데이터셋 생성
train_dataset = CustomDataset(os.path.join(data_dir, 'train'), transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

valid_dataset = CustomDataset(os.path.join(data_dir, 'valid'), transform=transform)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

# 사전 학습된 resnext50_32x4d 모델 사용
model = models.resnext50_32x4d(pretrained=True)

# 전이 학습을 위해 일부 가중치 고정 (freeze)
for param in model.parameters():
    param.requires_grad = False

# 출력 레이어를 분류하려는 클래스 수에 맞게 수정
model.fc = torch.nn.Sequential(
    torch.nn.Dropout(0.5),  # Dropout 추가
    torch.nn.Linear(model.fc.in_features, len(train_dataset.classes))
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 손실 함수 및 옵티마이저 설정
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.fc.parameters(), lr=0.001, momentum=0.9)

# 학습률 스케줄러 추가 (학습률 점진적 감소)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

# 학습 과정 (TQDM으로 진행 상황 시각화)
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    # TQDM으로 학습 진행 표시
    train_loader_iter = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")

    for images, labels in train_loader_iter:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        train_loader_iter.set_postfix(loss=running_loss / len(train_loader))

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

    # 학습률 스케줄러 업데이트
    scheduler.step()

# 모델 저장
torch.save(model.state_dict(), 'resnext_model.pth')

# 검증 평가
model.eval()
correct = 0
total = 0
with torch.no_grad():
    valid_loader_iter = tqdm(valid_loader, desc="Validating")  # 검증도 TQDM으로 진행 상황 시각화
    for images, labels in valid_loader_iter:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Validation Accuracy: {accuracy}%')


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNeXt50_32X4D_Weights.IMAGENET1K_V1`. You can also use `weights=ResNeXt50_32X4D_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnext50_32x4d-7cdf4587.pth" to /root/.cache/torch/hub/checkpoints/resnext50_32x4d-7cdf4587.pth
100%|██████████| 95.8M/95.8M [00:00<00:00, 155MB/s]
Epoch 1/10: 100%|██████████| 431/431 [03:14<00:00,  2.22it/s, loss=1.02]


Epoch 1/10, Loss: 1.0211076253804696


Epoch 2/10: 100%|██████████| 431/431 [03:15<00:00,  2.20it/s, loss=0.976]


Epoch 2/10, Loss: 0.9755381224050322


Epoch 3/10: 100%|██████████| 431/431 [03:14<00:00,  2.21it/s, loss=0.964]


...

Epoch 9/10, Loss: 0.9413373774552843


Epoch 10/10: 100%|██████████| 431/431 [03:15<00:00,  2.20it/s, loss=0.93]


Epoch 10/10, Loss: 0.9297795649747561


Validating: 100%|██████████| 54/54 [00:23<00:00,  2.33it/s]

Validation Accuracy: 55.8584686774942%


In [7]:
import os
import torch
import torchvision.models as models
import torchvision.transforms as T
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm  # TQDM import

# Custom dataset
class CustomDataset(Dataset): 
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = os.listdir(root_dir)
        self.data = []

        for label in range(len(self.classes)):
            class_folder = os.path.join(root_dir, self.classes[label])
            for filename in os.listdir(class_folder):
                img_path = os.path.join(class_folder, filename)
                self.data.append((img_path, label))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        image = Image.open(img_path).convert('RGB')  # 이미지 RGB로 변환
        if self.transform:
            image = self.transform(image)
        return image, label

# 경로 및 배치 크기 설정
data_dir = "."
batch_size = 32

# 데이터 증강 포함한 이미지 전처리
transform = T.Compose([
    T.Resize((224, 224)),
    T.RandomHorizontalFlip(),  # 랜덤 가로 뒤집기
    T.RandomRotation(10),  # 랜덤 회전
    T.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),  # 색상 변형
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 학습 및 검증 데이터셋 생성
train_dataset = CustomDataset(os.path.join(data_dir, 'train'), transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

valid_dataset = CustomDataset(os.path.join(data_dir, 'valid'), transform=transform)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

# 사전 학습된 resnext50_32x4d 모델 사용
model = models.resnext50_32x4d(pretrained=True)

# 모든 레이어 학습 가능하도록 설정 (고정 부분 제거)
# 이제 모든 파라미터가 학습에 참여함
for param in model.parameters():
    param.requires_grad = True

# 출력 레이어를 분류하려는 클래스 수에 맞게 수정
model.fc = torch.nn.Sequential(
    torch.nn.Dropout(0.5),  # Dropout 추가
    torch.nn.Linear(model.fc.in_features, len(train_dataset.classes))
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 손실 함수 및 옵티마이저 설정
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.00005, momentum=0.9)

# 학습률 스케줄러 추가 (학습률 점진적 감소)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

# 학습 과정 (TQDM으로 진행 상황 시각화)
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    # TQDM으로 학습 진행 표시
    train_loader_iter = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")

    for images, labels in train_loader_iter:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        train_loader_iter.set_postfix(loss=running_loss / len(train_loader))

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

    # 학습률 스케줄러 업데이트
    scheduler.step()

# 모델 저장
torch.save(model.state_dict(), 'resnext_model.pth')

# 검증 평가
model.eval()
correct = 0
total = 0
with torch.no_grad():
    valid_loader_iter = tqdm(valid_loader, desc="Validating")  # 검증도 TQDM으로 진행 상황 시각화
    for images, labels in valid_loader_iter:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Validation Accuracy: {accuracy}%')


Epoch 1/10: 100%|██████████| 431/431 [05:27<00:00,  1.32it/s, loss=1.08]


Epoch 1/10, Loss: 1.0825531928002696


Epoch 2/10: 100%|██████████| 431/431 [05:26<00:00,  1.32it/s, loss=0.962]


Epoch 2/10, Loss: 0.96152822006053


Epoch 3/10: 100%|██████████| 431/431 [05:26<00:00,  1.32it/s, loss=0.863]


...

Epoch 9/10, Loss: 0.6621346672145507


Epoch 10/10: 100%|██████████| 431/431 [05:26<00:00,  1.32it/s, loss=0.669]


Epoch 10/10, Loss: 0.66901260303234


Validating: 100%|██████████| 54/54 [00:23<00:00,  2.34it/s]

Validation Accuracy: 69.3155452436195%


In [2]:
import os
import torch
import torchvision.models as models
import torchvision.transforms as T
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from PIL import Image
from tqdm import tqdm  # TQDM import
import torch.nn as nn
from collections import Counter

# Custom dataset
class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = os.listdir(root_dir)
        self.data = []
        
        for label in range(len(self.classes)):
            class_folder = os.path.join(root_dir, self.classes[label])
            for filename in os.listdir(class_folder):
                img_path = os.path.join(class_folder, filename)
                self.data.append((img_path, label))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        image = Image.open(img_path).convert('RGB')  # 이미지 RGB로 변환
        if self.transform:
            image = self.transform(image)
        return image, label

# 경로 및 배치 크기 설정
data_dir = "."
batch_size = 32

# 데이터 증강 포함한 이미지 전처리
transform = T.Compose([
    T.Resize((224, 224)),
    T.RandomHorizontalFlip(),
    T.RandomRotation(15),  # 더 큰 회전 각도
    T.RandomAffine(degrees=0, translate=(0.1, 0.1)),  # 이미지 이동
    T.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.3),  # 색상 변형
    T.GaussianBlur(kernel_size=3),  # Gaussian Blur 추가
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 학습 및 검증 데이터셋 생성
train_dataset = CustomDataset(os.path.join(data_dir, 'train'), transform=transform)
valid_dataset = CustomDataset(os.path.join(data_dir, 'valid'), transform=transform)

# 각 클래스의 데이터 개수 계산 (Counter 사용)
class_counts = Counter([label for _, label in train_dataset])

# 클래스별로 가중치를 부여 (데이터 개수의 역수 사용)
class_weights = {cls: 1.0 / count for cls, count in class_counts.items()}

# 각 샘플의 가중치를 리스트로 변환
sample_weights = [class_weights[label] for _, label in train_dataset]

# WeightedRandomSampler 생성
sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights))

# WeightedRandomSampler를 적용한 DataLoader
train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=sampler)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

# 사전 학습된 resnext50_32x4d 모델 사용
from torchvision.models import ResNeXt50_32X4D_Weights

weights = ResNeXt50_32X4D_Weights.DEFAULT
model = models.resnext50_32x4d(weights=weights)

# 모든 레이어 학습 가능하도록 설정 (전이 학습 제외)
for param in model.parameters():
    param.requires_grad = True

# 출력 레이어를 분류하려는 클래스 수에 맞게 수정 (드롭아웃 비율 0.3으로 설정)
model.fc = nn.Sequential(
    nn.Dropout(0.3),  # Dropout 비율을 0.3으로 조정
    nn.Linear(model.fc.in_features, len(train_dataset.classes))
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Early Stopping 설정
best_accuracy = 0
patience = 5  # 개선되지 않는 에포크를 허용하는 최대 수
counter = 0

# 손실 함수 및 AdamW 옵티마이저 설정 (학습률 0.0001로 감소)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0001)

# 학습률 스케줄러 추가 (학습률 점진적 감소, gamma=0.5, step_size=10)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

# 학습 과정 (TQDM으로 진행 상황 시각화, 에포크 30)
num_epochs = 30
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    # TQDM으로 학습 진행 표시
    train_loader_iter = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")
    
    for images, labels in train_loader_iter:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        train_loader_iter.set_postfix(loss=running_loss / len(train_loader))
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")
    
    # 학습률 스케줄러 업데이트
    scheduler.step()

    # 검증 평가
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in valid_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Validation Accuracy: {accuracy}%')
    
    # Early Stopping 기준
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        counter = 0  # 성능 개선 시 카운터 초기화
        torch.save(model.state_dict(), 'best_model.pth')  # 최적의 모델 저장
    else:
        counter += 1
        if counter >= patience:
            print("Early stopping 적용")
            break

Epoch 1/30: 100%|████████████████████████████████████████████████████████| 863/863 [03:27<00:00,  4.16it/s, loss=0.761]


Epoch 1/30, Loss: 0.7608831753460032
Validation Accuracy: 69.77958236658932%


Epoch 2/30: 100%|████████████████████████████████████████████████████████| 863/863 [03:22<00:00,  4.26it/s, loss=0.626]


Epoch 2/30, Loss: 0.6255019191449652
Validation Accuracy: 70.70765661252901%


Epoch 3/30: 100%|█████████████████████████████████████████████████████████| 863/863 [03:21<00:00,  4.27it/s, loss=0.57]


...

Epoch 13/30, Loss: 0.2073700617208887
Validation Accuracy: 73.78190255220417%


Epoch 14/30: 100%|███████████████████████████████████████████████████████| 863/863 [03:17<00:00,  4.36it/s, loss=0.187]


Epoch 14/30, Loss: 0.18740442308013436
Validation Accuracy: 74.94199535962878%


Epoch 15/30: 100%|████████████████████████████████████████████████████████| 863/863 [03:17<00:00,  4.37it/s, loss=0.17]


Epoch 15/30, Loss: 0.1696912146303997
Validation Accuracy: 73.72389791183295%
Early stopping 적용


In [2]:
import os
import torch
import torchvision.models as models
import torchvision.transforms as T
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from PIL import Image
from tqdm import tqdm  # TQDM import
import torch.nn as nn
from collections import Counter

# Custom dataset
class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = os.listdir(root_dir)
        self.data = []
        
        for label in range(len(self.classes)):
            class_folder = os.path.join(root_dir, self.classes[label])
            for filename in os.listdir(class_folder):
                img_path = os.path.join(class_folder, filename)
                self.data.append((img_path, label))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        image = Image.open(img_path).convert('RGB')  # 이미지 RGB로 변환
        if self.transform:
            image = self.transform(image)
        return image, label

# 경로 및 배치 크기 설정
data_dir = "."
batch_size = 64

# 데이터 증강 포함한 이미지 전처리
transform = T.Compose([
    T.Resize((224, 224)),
    T.RandomHorizontalFlip(),
    T.RandomRotation(15),  # 더 큰 회전 각도
    T.RandomAffine(degrees=0, translate=(0.1, 0.1)),  # 이미지 이동
    T.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.3),  # 색상 변형
    T.GaussianBlur(kernel_size=3),  # Gaussian Blur 추가
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 학습 및 검증 데이터셋 생성
train_dataset = CustomDataset(os.path.join(data_dir, 'train'), transform=transform)
valid_dataset = CustomDataset(os.path.join(data_dir, 'valid'), transform=transform)

# 각 클래스의 데이터 개수 계산 (Counter 사용)
class_counts = Counter([label for _, label in train_dataset])

# 클래스별로 가중치를 부여 (데이터 개수의 역수 사용)
class_weights = {cls: 1.0 / count for cls, count in class_counts.items()}

# 각 샘플의 가중치를 리스트로 변환
sample_weights = [class_weights[label] for _, label in train_dataset]

# WeightedRandomSampler 생성
sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights))

# WeightedRandomSampler를 적용한 DataLoader
train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=sampler)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

# 사전 학습된 resnext50_32x4d 모델 사용
from torchvision.models import ResNeXt50_32X4D_Weights

weights = ResNeXt50_32X4D_Weights.DEFAULT
model = models.resnext50_32x4d(weights=weights)

# 모든 레이어 학습 가능하도록 설정 (전이 학습 제외)
for param in model.parameters():
    param.requires_grad = True

# 출력 레이어를 분류하려는 클래스 수에 맞게 수정 (드롭아웃 비율 0.3으로 설정)
model.fc = nn.Sequential(
    nn.Dropout(0.3),  # Dropout 비율을 0.3으로 조정
    nn.Linear(model.fc.in_features, len(train_dataset.classes))
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Early Stopping 설정
best_accuracy = 0
patience = 30  # 개선되지 않는 에포크를 허용하는 최대 수
counter = 0

# 손실 함수 및 AdamW 옵티마이저 설정 (학습률 0.0001로 감소)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0001)

# 학습률 스케줄러 추가 (학습률 점진적 감소, gamma=0.5, step_size=10)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

# 학습 과정 (TQDM으로 진행 상황 시각화, 에포크 30)
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    # TQDM으로 학습 진행 표시
    train_loader_iter = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")
    
    for images, labels in train_loader_iter:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        train_loader_iter.set_postfix(loss=running_loss / len(train_loader))
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")
    
    # 학습률 스케줄러 업데이트
    scheduler.step()

    # 검증 평가
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in valid_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Validation Accuracy: {accuracy}%')
    
    # Early Stopping 기준
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        counter = 0  # 성능 개선 시 카운터 초기화
        torch.save(model.state_dict(), 'best_model.pth')  # 최적의 모델 저장
    else:
        counter += 1
        if counter >= patience:
            print("Early stopping 적용")
            break

# 베스트 모델의 혼동 행렬 시각화 및 성능 출력
if best_cm is not None:
    plt.figure(figsize=(8, 6))
    sns.heatmap(best_cm, annot=True, fmt='d', cmap='Blues', xticklabels=train_dataset.classes, yticklabels=train_dataset.classes)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title(f'Best Model Confusion Matrix\nValidation Accuracy: {best_accuracy:.3f}%')
    plt.show()


Epoch 1/100: 100%|███████████████████████████████████████████████████████| 216/216 [07:55<00:00,  2.20s/it, loss=0.791]


Epoch 1/100, Loss: 0.7910682476229138
Validation Accuracy: 69.60556844547564%


Epoch 2/100: 100%|███████████████████████████████████████████████████████| 216/216 [08:07<00:00,  2.26s/it, loss=0.593]


Epoch 2/100, Loss: 0.5930374179173399
Validation Accuracy: 69.66357308584686%


Epoch 3/100: 100%|███████████████████████████████████████████████████████| 216/216 [07:33<00:00,  2.10s/it, loss=0.539]


...

Epoch 43/100, Loss: 0.020644812814242432
Validation Accuracy: 75.0%


Epoch 44/100: 100%|█████████████████████████████████████████████████████| 216/216 [03:44<00:00,  1.04s/it, loss=0.0196]


Epoch 44/100, Loss: 0.019602143966444094
Validation Accuracy: 75.92807424593967%


Epoch 45/100: 100%|█████████████████████████████████████████████████████| 216/216 [05:27<00:00,  1.51s/it, loss=0.0164]


Epoch 45/100, Loss: 0.01635756100491832
Validation Accuracy: 74.82598607888632%
Early stopping 적용
